In [1]:
import torchvision
import torch

import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class Sqeeze(object):
    def __call__(self, sample):
        sqzd = torch.squeeze(sample)
        return sqzd
    
    def __repr__(self) -> str:
        return 'Squeze()'


In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081)),
    Sqeeze()
])

mnist_train = torchvision.datasets.MNIST(root='./dataset', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='./dataset', train=False, download=True, transform=transform)

In [4]:
mnist_train_data = mnist_train.data
mnist_train_labels = mnist_train.targets
mnist_test_data = mnist_test.data
mnist_test_labels = mnist_test.targets

In [5]:
mnist_train_data.shape

torch.Size([60000, 28, 28])

In [6]:
mnist_train_data_0 = mnist_train_data[mnist_train_labels == 0]
mnist_train_data_1 = mnist_train_data[mnist_train_labels == 1]
mnist_train_data_2 = mnist_train_data[mnist_train_labels == 2]

mnist_test_data_0 = mnist_test_data[mnist_test_labels == 0]
mnist_test_data_1 = mnist_test_data[mnist_test_labels == 1]
mnist_test_data_2 = mnist_test_data[mnist_test_labels == 2]

mnist_train_labels_0 = mnist_train_labels[mnist_train_labels == 0]
mnist_train_labels_1 = mnist_train_labels[mnist_train_labels == 1]
mnist_train_labels_2 = mnist_train_labels[mnist_train_labels == 2]

mnist_test_labels_0 = mnist_test_labels[mnist_test_labels == 0]
mnist_test_labels_1 = mnist_test_labels[mnist_test_labels == 1]
mnist_test_labels_2 = mnist_test_labels[mnist_test_labels == 2]

In [7]:
mnist_train_data = np.vstack([mnist_train_data_0, mnist_train_data_1, mnist_train_data_2])
mnist_train_labels = np.hstack([mnist_train_labels_0, mnist_train_labels_1, mnist_train_labels_2])
# mnist_train_labels = (mnist_train_labels == 8) + 0

mnist_test_data = np.vstack([mnist_test_data_0, mnist_test_data_1, mnist_test_data_2])
mnist_test_labels = np.hstack([mnist_test_labels_0, mnist_test_labels_1, mnist_test_labels_2])
# mnist_test_labels = (mnist_test_labels == 8) + 0

In [8]:
mnist_train_labels

array([0, 0, 0, ..., 2, 2, 2])

In [9]:
from torch.utils.data import TensorDataset

mnist_train_data, mnist_train_labels = torch.tensor(mnist_train_data).to(device), torch.tensor(mnist_train_labels).to(device)
mnist_test_data, mnist_test_labels = torch.tensor(mnist_test_data).to(device), torch.tensor(mnist_test_labels).to(device)

mnist_train_data = torchvision.transforms.Normalize((0.1307,), (0.3081))(mnist_train_data.float())
mnist_test_data = torchvision.transforms.Normalize((0.1307,), (0.3081))(mnist_test_data.float())

mnist_train = TensorDataset(mnist_train_data, mnist_train_labels)
mnist_test = TensorDataset(mnist_test_data, mnist_test_labels)

In [10]:
train_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(mnist_test, batch_size=32, shuffle=True)

In [11]:
for input, label in test_dataloader:
    print(input.shape)
    print(input.dtype)
    print(label.shape)
    print(label.dtype)
    break

torch.Size([32, 28, 28])
torch.float32
torch.Size([32])
torch.int64


In [12]:
import os
from main import run_model, init_model_params
from datetime import datetime
from models import GCNAuto

model_name = 'imagine_gcn_auto'
random_seed = 0
dataset_name = 'mnist'
time_now = datetime.now().strftime('%Y-%m-%d-%H-%M')
results_path = os.path.join('output', time_now, model_name, dataset_name, str(random_seed))
os.makedirs(results_path, exist_ok=True)

model = GCNAuto(in_features=28, n_nodes=28, num_classes=3, hidden_sizes=[256, 512, 256], dropout_p=0.2, device=device)

dataset_sizes = {'train': len(train_dataloader), 'val': len(test_dataloader), 'test': len(test_dataloader)}
dataloaders = {'train': train_dataloader, 'val': test_dataloader, 'test': test_dataloader}
class_names = None

In [13]:
print(dataset_sizes)
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'val']}
print(dataset_sizes)

{'train': 582, 'val': 99, 'test': 99}
{'train': 582, 'val': 99}


In [14]:
model = init_model_params(model, random_seed=random_seed)
model.init_node_embeddings()
model = model.to(device)

In [15]:
run_model(random_seed, dataloaders, dataset_sizes, class_names, model, results_path)

train() called: model=GCNAuto, opt=Adam(lr=0.001000), epochs=2, device=cuda

Epoch   1/  2, train loss:  6.55, train acc:  0.97, val loss:  0.81, val acc:  0.98

Time total:      4.48 sec
Time per epoch:  2.24 sec
[0 1 2]
output/2022-01-07-11-47/imagine_gcn_auto/mnist/0


In [16]:
mnist_test_data.shape

torch.Size([3147, 28, 28])

In [17]:
model.eval()
with torch.no_grad():
    outputs = model(mnist_test_data)
    _, y_preds = torch.max(outputs, 1)


In [18]:
from sklearn.metrics import classification_report

In [19]:
cr = classification_report(mnist_test_labels.cpu().numpy(), y_preds.cpu().numpy())

In [20]:
print(cr)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      0.99      0.99      1135
           2       0.98      0.99      0.99      1032

    accuracy                           0.99      3147
   macro avg       0.99      0.99      0.99      3147
weighted avg       0.99      0.99      0.99      3147



# New training func

In [21]:
import time

def train(model, optimizer, loss_fn, train_dl, val_dl, epochs=100, device='cpu'):

    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    history = {} # Collects per-epoch loss and acc like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    history['acc'] = []
    history['val_acc'] = []

    start_time_sec = time.time()

    for epoch in range(1, epochs+1):

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        model.train()
        train_loss         = 0.0
        num_train_correct  = 0
        num_train_examples = 0

        for batch in train_dl:

            optimizer.zero_grad()

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            loss = loss_fn(yhat, y)

            loss.backward()
            optimizer.step()

            train_loss         += loss.data.item() * x.size(0)
            num_train_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_train_examples += x.shape[0]

        train_acc   = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        val_loss       = 0.0
        num_val_correct  = 0
        num_val_examples = 0

        for batch in val_dl:

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            loss = loss_fn(yhat, y)

            val_loss         += loss.data.item() * x.size(0)
            num_val_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_val_examples += y.shape[0]

        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_dl.dataset)


        if epoch == 1 or epoch % 10 == 0:
          print('Epoch %3d/%3d, train loss: %5.2f, train acc: %5.2f, val loss: %5.2f, val acc: %5.2f' % \
                (epoch, epochs, train_loss, train_acc, val_loss, val_acc))

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['acc'].append(train_acc)
        history['val_acc'].append(val_acc)

    # END OF TRAINING LOOP


    end_time_sec       = time.time()
    total_time_sec     = end_time_sec - start_time_sec
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))

    return history

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, PARAMS['SCHEDULER_STEPSIZE'], PARAMS['SCHEDULER_GAMMA'])

history = train(
    model = model,
    optimizer = optimizer,
    loss_fn = criterion,
    train_dl = train_dataloader,
    val_dl = test_dataloader,
    device='cuda')

train() called: model=GCNAuto, opt=Adam(lr=0.001000), epochs=100, device=cuda

Epoch   1/100, train loss:  1.18, train acc:  0.98, val loss:  0.39, val acc:  0.99
Epoch  10/100, train loss:  0.05, train acc:  0.99, val loss:  0.15, val acc:  0.99
Epoch  20/100, train loss:  0.07, train acc:  0.99, val loss:  0.03, val acc:  0.99
Epoch  30/100, train loss:  0.05, train acc:  0.99, val loss:  0.10, val acc:  0.99
Epoch  40/100, train loss:  0.10, train acc:  0.98, val loss:  0.04, val acc:  0.99
Epoch  50/100, train loss:  0.05, train acc:  0.99, val loss:  0.09, val acc:  0.98
Epoch  60/100, train loss:  0.12, train acc:  0.97, val loss:  0.12, val acc:  0.97
Epoch  70/100, train loss:  0.10, train acc:  0.98, val loss:  0.11, val acc:  0.99
Epoch  80/100, train loss:  0.28, train acc:  0.94, val loss:  0.20, val acc:  0.98
Epoch  90/100, train loss:  0.14, train acc:  0.97, val loss:  0.11, val acc:  0.99
Epoch 100/100, train loss:  0.30, train acc:  0.91, val loss:  0.56, val acc:  0.